In [10]:
!pip install fastapi uvicorn langchain langchain-openai langchain-teddynote python-dotenv 

  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/79.5 MB ? eta -:--:--
     - -------------------------------------- 3.1/79.5 MB 15.3 MB/s eta 0:00:05
     -- ------------------------------------- 5.2/79.5 MB 12.7 MB/s eta 0:00:06
     --- ------------------------------------ 7.9/79.5 MB 12.5 MB/s eta 0:00:06
     ---- ----------------------------------- 9.4/79.5 MB 11.5 MB/s eta 0:00:07
     ----- --------------------------------- 10.7/79.5 MB 10.5 MB/s eta 0:00:07
     ----- ---------------------------------- 11.8/79.5 MB 9.5 MB/s eta 0:00:08
     ------ --------------------------------- 1

In [2]:
!uvicorn main:app --reload

^C


In [ ]:
# 기본 소스
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from fastapi import FastAPI
from dotenv import load_dotenv
import os


load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

app = FastAPI()

llm = ChatOpenAI(
   temperature = 0.1,  # 창의성
   model_name = 'gpt-4o-mini',  # 모델명
)

question = "대한민국의 수도는 어디인가요?"
response = llm.invoke(question).content
response

'대한민국의 수도는 서울입니다.'

In [8]:
# LogProb 활성화
# 주어진 텍스트에 대한 모델의 토큰 확률의 로그 값
# 토큰이란 문장을 구성하는 개별 단어나 문자 등의 요소를 의미
# 토큰 확률은 모델이 그 토큰을 예측할 확률

llm_with_logprob = ChatOpenAI(
   temperature = 0.1,  # 창의성
   max_tokens = 2048,  # 최대 토큰수
   model_name = 'gpt-4o-mini',  # 모델명
).bind(logprobs=True)

question = "대한민국의 수도는 어디인가요?"
response = llm_with_logprob.invoke(question).content
response

'대한민국의 수도는 서울입니다.'

In [9]:
# 스트리밍 출력
# 스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용

# 스트림 방식으로 질의
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요")

for token in answer:
   print(token.content, end="", flush = True)

대한민국에는 아름다운 관광지가 많이 있습니다. 아래는 추천하는 10곳과 그 주소입니다.

1. **경복궁**
   - 주소: 서울특별시 종로구 사직로 161

2. **제주도**
   - 주소: 제주특별자치도 제주시

3. **부산 해운대**
   - 주소: 부산광역시 해운대구 해운대해변로 140

4. **경주 불국사**
   - 주소: 경상북도 경주시 불국로 385

5. **남이섬**
   - 주소: 강원도 춘천시 남이섬길 1

6. **전주 한옥마을**
   - 주소: 전라북도 전주시 완산구 기린대로 99

7. **설악산 국립공원**
   - 주소: 강원도 속초시 설악산로 173

8. **안동 하회마을**
   - 주소: 경상북도 안동시 풍천면 하회리

9. **서울 남산타워 (N서울타워)**
   - 주소: 서울특별시 용산구 남산공원길 105

10. **광주 무등산**
    - 주소: 광주광역시 동구 무등산로 100

각 관광지는 독특한 매력을 가지고 있으며, 방문 시 다양한 문화와 자연을 경험할 수 있습니다.

In [ ]:
# 멀티모달 모델(이미지 인식)
# 여러 가지 형태의 정보(모달)를 통합하여 처리하는 기술이나 접근 방식
# 텍스트, 이미지, 오디오, 비디오

from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

llm = ChatOpenAI(
   temperature = 0.1,
   max_tokens = 2048,
   model_name="gpt-4o"
)

multimodal_llm = MultiModal(llm)

# image_url = "..."                                # 샘플이미지 주소(웹사이트로 부터 바로 인식)
# answer = multimodal_llm.stream(image_url)
# stream_response(answer)

image_file_path = "./sample/sample_image.jpg"      # 로컬 PC에 저장되어 있는 이미지의 경로 입력
answer = multimodal_llm.stream(image_file_path)    # 이미지 파일로부터 질의 (스트림 방식)
stream_response(answer)                            # 스트리밍 방식으로 각 토큰을 실시간 출력


In [ ]:
# system, user 프롬프트 수정
system_prompt = """너는 이미지를 분석해주는 AI 어시스턴트야."""
user_prompt = """너에게 주어지는 이미지를 흥미롭게 정리해서 답변해줘"""

multimodal_llm_with_prompt = MultiModal(
   llm, system_prompt = system_prompt, user_prompt = user_prompt
)

# 로컬에 저장되어 있는 이미지 경로 입력
answer = multimodal_llm_with_prompt.stream(image_file_path)

# 스트리밍 방식으로 각 토큰을 실시간 출력
stream_response(answer)